In [ ]:
!pip install unsloth
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from unsloth import FastLanguageModel
import torch
import copy

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# load model từ hf với độ dài ngữ cảnh là 2048, và nén lại ở 4 bit
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.11.1: Fast Gemma2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
temp = copy.deepcopy(model)

# chèn các matrix vào các layer
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.11.1 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [ ]:
# load dataset
from datasets import load_dataset


dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/train_math_text_flexible.jsonl",
    split="train"
)


print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': 'Hãy diễn giải công thức dưới đây bằng lời dễ hiểu.', 'input': 'a + b = b + a', 'output': 'Tổng hai số a và b bằng tổng b và a.'}


In [ ]:
# format các mẫu dữ liệu trước khi huấn luyện
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]

    texts = []
    for instruction, input_text, output_text in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input_text, output_text) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


train_test = dataset.train_test_split(
    test_size=0.1,
    seed=42
)

train_ds = train_test["train"]
eval_ds  = train_test["test"]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset[10]["text"])


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Giải thích ý nghĩa của công thức này bằng tiếng Việt.

### Input:
f'(x) = lim(h→0) [f(x+h) - f(x)] / h

### Response:
Đạo hàm của hàm số được xác định bằng giới hạn của hiệu thương khi h tiến tới 0.
<eos>


In [ ]:
# test trước khi chừa fine-tune
FastLanguageModel.for_inference(temp)
inputs = tokenizer(
[
    prompt.format(
        "Viết lại công thức toán học sau bằng ngôn ngữ tự nhiên.",
        "(a + b)(a - b) = a^2 - b^2",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = temp.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, eos_token_id=tokenizer.eos_token_id)

<bos>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Viết lại công thức toán học sau bằng ngôn ngữ tự nhiên.

### Input:
(a + b)(a - b) = a^2 - b^2

### Response:

(a + b)(a - b) = a^2 - b^2

### Explanation:

(a + b)(a - b) = a^2 - b^2

### Explanation:

(a + b)(a - b) = a^2 - b^2

### Explanation:

(a + b)(a - b) = a^2 - b^2

### Explanation:

(a + b)(a - b) = a^2 - b^2

### Explanation:

(a + b)(a - b) = a^2 - b^2




In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience = 3,

    early_stopping_threshold = 0.0,


)

args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        fp16_full_eval = True,
        per_device_eval_batch_size = 2,
        eval_accumulation_steps = 4,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        logging_steps = 10,
        report_to = "none",
        output_dir = "training_checkpoints",
        save_strategy = "steps",
        save_steps = 10,
        save_total_limit = 3,
        eval_strategy = "steps",
        eval_steps = 10,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = eval_ds,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = args,
    callbacks=[early_stopping_callback],
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/900 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 1 | Total steps = 113
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)


Step,Training Loss,Validation Loss
10,0.461100,0.336954
20,0.085300,0.101982
30,0.073200,0.078332
40,0.057900,0.070020
50,0.061500,0.068816
60,0.056800,0.067897
70,0.054600,0.067559
80,0.054700,0.066168
90,0.119400,0.065438
100,0.057100,0.065029


Unsloth: Not an error, but Gemma2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
# test lại sau khi huấn luyện
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "Viết lại công thức toán học sau bằng ngôn ngữ tự nhiên.",
        "(a + b)(a - b) = a^2 - b^2",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512, eos_token_id=tokenizer.eos_token_id)

<bos>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Viết lại công thức toán học sau bằng ngôn ngữ tự nhiên.

### Input:
(a + b)(a - b) = a^2 - b^2

### Response:

Tích của tổng và hiệu hai số bằng hiệu hai bình phương của chúng.
<eos>


In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/training_checkpoints/checkpoint-110",
    max_seq_length = 2028,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Gemma2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.1 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [ ]:
# !curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# model.save_pretrained_gguf("equation_text", tokenizer, quantization_method = "f16")

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [01:20<00:00, 80.42s/it]


Unsloth: Merge process complete. Saved to `/content/equation_text`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: llama.cpp folder exists but binaries not found - will rebuild
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Install GGUF and other packages


In [ ]:
# # tự động merge và đẩy lên hf
# model.push_to_hub_merged(
#     tokenizer=tokenizer,
#     repo_id="trongdung143/equation_text",
#     token="",

# )